# Modelos Latentes

Las calificaciones que los usuarios realizan de los items, en nuestro caso peliculas, aportan valiosa informacion sobre las preferencias de los mismos. Por ejemplo podemos observar que un usuario que ha calificado muy bien a Star Wars, Star Trek y Superman probablemente sea fanatico de ciencia ficcion y por lo tnato deberiamos estimar una calificacion alta para este tipo de peliculas.

El algoritmo de <b>collaborative filtering</b> se limita unicamente a calcular semejanzas entre peliculas pero no es capaz de darse cuenta que un grupo de peliculas similares expresa la actitud del usuario hacia las peliculas de dicho tipo. A este tipo de modelizacion la llamamos <b>modelo latente</b> porque nos permite inferir <b>preferencias de los usuarios que no estan explicitas en la matriz de utilidad</b>.

Hay dos metodos que funcionan muy bien para obtener la informacion latente de la matriz de utilidad: <b>RBMs</b> o <b>SVD++</b>.

## SVD++

El algoritmo <b>SVD++</b> fue presentado informalmente durante la competencia del premio <b>Netflix</b> por un millon de usd. <b>Este algoritmo represento un importante salto en la mejora de los sistemas de recomendaciones</b> y hoy en dia se usa extensamente. Esta basado en el simple concepto de factorizar la matriz de utilidad para descubrir los modelos latentes.

El principio aqui es muy simple: descomponer la matriz de utilidad en el producto de dos matrices: <b>Q</b> y <b>P</b>. En donde <b>Q</b> tendra tantas <b>filas</b> como <b>items</b> y tantas <b>columnas</b> como <b>factores latentes</b> usemos y <b>P</b> tendra tantas <b>filas</b> como <b>usuarios</b> y tantas <b>columnas</b> como <b>factores</b>.

<img src="imagenes/descomposicion_Q_P.png">

La cantidad de factores <b>r</b> es un parametro que debemos fijar. Al descomponer la matriz <b>Q</b> nos dara la relacion entre los <b>items</b> y los <b>factores</b>. <b>P</b> nos dara la relacion entre los <b>usuarios</b> y los <b>factores</b>. Es decir que en Q vamos a poder observar que tipo de factores estan asociados a cada pelicula y en P tendremos las preferencias de los usuarios con respecto a los factores.

<b>El objetivo es encontrar Q y P tales que Q * R minimice el error con la matriz de utilidad original.</b>

Si logramos hallar estas matrices Q y P entonces podemos estimar la calificacion del usuario <b>i</b> para la pelicula <b>j</b> haciendo <b>q sub i</b> * <b>p sub j</b>. Notar que para las calificaciones que ya teniamos esto deberia darnos un numero muy similar al de la matriz de utilidad ya que hemos construido a P y Q para minimizar la diferencia entre QP y la matriz original. Lo interesante de esto es que las matrices Q y P que minimicen el error con la original nos dan magicamente todos los valores que nos faltaban en la matriz de utilidad original. 

Por <b>ejemplo</b> en nuestro caso si queremos estimar la calificacion del usuario 5 para el item 2. Hacemos q2 = (-0.5,0.6,0.5) y p5=(-2,0.3,2.4), <b>p2 * q5t = 2.4</b>.

<img src="imagenes/ejemplo_descomposicion_Q_P.png">

Tenemos ahora que analizar de que forma <b>hallar</b> las matrices Q y P.

Notamos que el problema es muy similar a la indexacion semantica latente en donde usamos la descomposicion SVD para reducir las dimensiones de una matriz y obtener un modelo latente. Desafortunadamente no podemos usar SVD para nuestro sistema de recomendaciones porque la matriz de utilidad no esta completa, tiene muchos "huecos". Lamentablemente SVD no funciona si no tenemos odos los datos de la matriz original es por eso que necesitamos un algorimo similar a SVD pero que funciones con matrices imcompletas. Algo asi como una SVD parcial.

Podemos escribir el siguiente problema de optimizacion:

<img src="imagenes/problema_de_optimizacion.png">

En donde la matriz <b>r</b> que usamos es nuestro set de entranamiento, es decir el 80% de los datos de nuestro sistema.

Aqui tenemos que detenernos a pensar porque pese a que aun no tenemos el algoritmo ya tenemos un potencial problema con el mismo. Podriamos encontrar valores para las mastrices Q Y P que minimicen la diferencia con la matriz de utilidad de nuestro set de entrenamiento pero que funcone muy mal para el set de pruebas (el otro 20% que reservamos para evaluar el sistema). Esto es un clasico problema de overfitting en el cual nuestro algoritmo aprende a ajustar muy bien al set de entrenamiento pero funciona muy mal para realizar predicciones.

La forma de solucionar el overfitting es intentar minimizar las normas de las matrices Q y P. A esto sel o llama <b>regularizacion</b>. Nuestro problema es entonces:

<img src="imagenes/problema_de_optimizacion_regularizado.png">

Donde lamda 1 y lamda 2 son <b>hiperparametros</b> a definir.

Como en tantos otros problemas de optimizacion podemos encontrar la solucion mediante <b>Gradient Descent</b>. Concretamente usaremos la variante <b>Stochastic Gradient Descent</b> en donde vamos a ir optimizando P y Q en base a cada una de las calificaciones, que ya conocemos, procesando estas calificaciones una por una.

<b>Algoritmo de Gradient Descent para calcular P y Q.

<img src="imagenes/algoritmo_SVD++.png">

Este algoritmo es eficiente y funciona muy aceptablemente. Por lo que hemos obtenido una solucion muy eficiente para estimar las calificaciones que nos faltan en nuestro sistema en base al modelo latente. Como se podran imaginar estamos muy lejos de haber terminado.

Notar que los parametros lamda no se optimizan en el algoritmo. A los parametro que no se optimizan dentro del algoritmo los llamamos "hiper-parametros". Los "hiper-parametros" en general se buscan realizando un grid-search en base a una serie de valores posibles. Por ejemplo podemos probar con lambda = {0.005,0.05,0.5,0,0.5,1.2} y entonces tenemos 36 combinaciones a probar para nustros dos valores de lambda. Corremos el algoritmo de optimizacion SGD para cada valor y nos quedamos con los lambda que minimicen el error.

### Agregando las desviaciones del usuario y de la pelicula

Podemos volver a pensar que es conveniente modelar las recomendaciones en base a desviaciones a partir del promedio de calificaciones global:

<img src="imagenes/recomendacion_con_desviacion_calif_global.png">

Sabemos como calcular Q y P, ahora QP nos dara una desviacion en lugar de una recomendacion pero el algoritmo es exactamente el mismo. Mu es el promedio global de todas las calificaciones. Y bx y bi ahora son valores que podemos tambien aprender en lugar de calcularlos.